In [10]:
pip install scipy


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Context

Kirk's approximation is an analytical formula for valuing a call option on a spread between two assets (in this case: UK & FR day-ahead prices). In our context:
- S_1 = UK Forward Price
- S_2 = FR Forward Price
- Sigma_1, Sigma_2 = volatilities of UK and FR
- p (rho) = correlation between them
- K = strike price (capacity cost, often 0 in our case)
- T = time to expiry in years

A notable benefit is that it circumvents the computational overhead associated with Monte Carlo simulations.

The output will be EUR per MWh option value, which you can then multiply by capacity to get a notional market value for transmission rights.


In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import sys
import os
sys.path.append(os.path.abspath("/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main"))
from config import raw_data_dir, processed_data_dir, API_KEY, FR_DOMAIN, START_DATE, END_DATE, FX_GBP_EUR

In [12]:
# Load merged hourly dataset 
data_path = os.path.join(processed_data_dir, "UK_FR_day_ahead_hourly_merged_spread.csv")
df = pd.read_csv(data_path, parse_dates=["datetime"])

In [13]:
# Ensure sorted and clean
df = df.sort_values("datetime").reset_index(drop=True)


In [14]:
S1 = df["UK_price_eur"].mean()  # Mean UK price in EUR/MWh
S2 = df["FR_price"].mean()      # Mean FR price in EUR/MWh

# From return-based volatilities (already annualised in your EDA)
sigma1 = 0.17 #EUR/MWh
sigma2 = 0.48 #EUR/MWh
rho = 0.106   # Pearson correlation from returns

In [15]:
print(S1, S2)
print(sigma1)
print(sigma2)
print(rho)

81.73859201229604 77.87693781035706
0.17
0.48
0.106


Volatility is first measured on an hourly basis and then scaled using the √T rule, where T is the time horizon in hours.  
Annualising (σ × √8760) is standard convention for option models, but you can also scale to shorter horizons (e.g. 1M, 6M) — the key is that variance grows with time while standard deviation grows with the square root of time. 

Usually don’t strictly need a 1Y (or 2Y) of history to compute these, but the estimate is more reliable when the data window matches or exceeds the horizon since shorter samples can be regime specific and noisier.

In [16]:
K = 0 # Strike price (capacity cost) in EUR/MWh
T = 1/12     # 1 month to expiry

In [17]:
# Kirk's Approximation (#source used: Kirk Formula and Modified Kirk Formula for Spread Option Pricing in Python - Statistics and Risk Modelling - YT)
b = S2 / (S2 + K)
sigma_k = np.sqrt(sigma1**2 - 2*b*rho*sigma1*sigma2 + (b**2)*(sigma2**2))
d1 = (np.log(S1 / (S2 + K)) + 0.5 * sigma_k**2 * T) / (sigma_k * np.sqrt(T))
d2 = d1 - sigma_k * np.sqrt(T)
C = S1 * norm.cdf(d1) - (S2 + K) * norm.cdf(d2) #Where N - (norm.cdf) is the standard normal CDF, C is the price of the spread option

In [18]:
print(f"UK mean price (S1): {S1:.2f} EUR/MWh")
print(f"FR mean price (S2): {S2:.2f} EUR/MWh")
print(f"UK vol (annualised): {sigma1:.2f}")
print(f"FR vol (annualised): {sigma2:.2f}")
print(f"Correlation: {rho:.3f}")
print(f"Spread Option Value (1M, Kirk's): {C:.2f} EUR/MWh")

UK mean price (S1): 81.74 EUR/MWh
FR mean price (S2): 77.88 EUR/MWh
UK vol (annualised): 0.17
FR vol (annualised): 0.48
Correlation: 0.106
Spread Option Value (1M, Kirk's): 6.71 EUR/MWh
